# الگوریتم مقایسه شباهت (Levenshtein/SequenceMatcher)  20%

In [1]:
import re
from difflib import get_close_matches

# مرحله ۱: لود کردن دیکشنری لغات از فایل
with open("/content/Vocabulary.txt", "r") as f:
    vocabulary = set(word.strip().lower() for word in f if word.strip())

# مرحله ۲: جمله‌ی دارای غلط املایی
# The quick brown fox jumps over the lazy dog.
misspelled_sentence = "Th quik braown foxs jmups ovem te lagzy qog."

# مرحله ۳: توکنایز جمله به کلمات
words = re.findall(r"\b\w+\b", misspelled_sentence.lower())

# تابع اصلاح غلط‌های املایی
def correct_word(word, vocab):
    matches = get_close_matches(word, vocab, n=1, cutoff=0.8)
    return matches[0] if matches else word

# اصلاح تمام کلمات جمله
corrected_words = [correct_word(word, vocabulary) for word in words]

# بازسازی جمله اصلاح‌شده
corrected_sentence = " ".join(corrected_words).capitalize() + "."

# نمایش نتیجه‌ها
print("\nOriginal sentence:")
print("The quick brown fox jumps over the lazy dog.")
print("\nMisspelled sentence:")
print(misspelled_sentence)
print("\nCorrected sentence:")
print(corrected_sentence)



Original sentence:
The quick brown fox jumps over the lazy dog.

Misspelled sentence:
Th quik braown foxs jmups ovem te lagzy qog.

Corrected sentence:
The quirk brown foxes mumps ovem te lazy qog.


In [7]:
import difflib
import re

# Load the vocabulary from the uploaded file
with open("/content/Vocabulary.txt", "r") as file:
    vocabulary = set(word.strip().lower() for word in file.readlines() if word.strip())

# Input: Misspelled sentence
misspelled_sentence = "Th quik braown foxs jmups ovem te lagzy qog."

# Improved tokenizer (handles punctuation better)
def tokenize(sentence):
    return re.findall(r"\b\w+\b|[^\w\s]", sentence)

# Function to find the most similar word using difflib with a higher cutoff
def correct_word(word, vocab, cutoff=0.85):
    word = word.lower()
    matches = difflib.get_close_matches(word, vocab, n=1, cutoff=cutoff)
    return matches[0] if matches else word

# Correct the sentence
def correct_sentence(sentence, vocab):
    tokens = tokenize(sentence)
    corrected_tokens = []
    for token in tokens:
        if re.match(r"\w+", token):  # If it's a word
            corrected = correct_word(token, vocab)
            corrected_tokens.append(corrected)
        else:
            corrected_tokens.append(token)  # Keep punctuation as is
    return ' '.join(corrected_tokens).replace(" .", ".")

corrected_sentence = correct_sentence(misspelled_sentence, vocabulary)
corrected_sentence


'th quirk brown foxes jmups ovem te lazy qog.'

# الگوریتم اصلاح‌کننده‌ی نورویگ (Norvig Spell Corrector) 20%
###فقط بر اساس شباهت نگارشی و احتمال آماری عمل می‌کنه، بدون درک معنی جمله

In [10]:
import re
from collections import Counter
import math

# Load vocabulary again to ensure clean state
with open("/content/Vocabulary.txt", "r") as f:
    vocabulary = set(word.strip().lower() for word in f.readlines() if word.strip())

# Create word frequency using vocabulary (simulate frequency)
word_freq = Counter({word: 1 for word in vocabulary})

# Norvig’s original probability-based spell corrector

def words(text):
    return re.findall(r'\w+', text.lower())

def P(word, N=sum(word_freq.values())):
    return word_freq[word] / N

def correction(word):
    return max(candidates(word), key=P)

def candidates(word):
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words_set):
    return set(w for w in words_set if w in vocabulary)

def edits1(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

# Apply Norvig-based corrector on sentence
def norvig_correct_sentence(sentence):
    tokens = re.findall(r"\b\w+\b|[^\w\s]", sentence)
    corrected = []
    for token in tokens:
        if token.isalpha():
            corrected.append(correction(token.lower()))
        else:
            corrected.append(token)
    return ' '.join(corrected).replace(" .", ".")

# Test sentence with errors
test_sentence = "Th quik braown foxs jmups ovem te lagzy qog."
norvig_result = norvig_correct_sentence(test_sentence)
norvig_result


'tb quip brawn foxy jumps ovum te lazy tog.'

# SymSpell 40% الگوریتم -

In [12]:
!pip install symspellpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 kB 8.9 MB/s eta 0:00:00


In [16]:
from symspellpy.symspellpy import SymSpell, Verbosity
import re

# Initialize SymSpell
max_edit_distance_dictionary = 2
prefix_length = 7
sym_spell = SymSpell(max_edit_distance_dictionary, prefix_length)

# Load dictionary
sym_spell.load_dictionary("/content/Vocabulary.txt", term_index=0, count_index=1)

# تصحیح جمله
def symspell_correct_sentence(sentence):
    tokens = re.findall(r"\b\w+\b|[^\w\s]", sentence)
    corrected = []
    for token in tokens:
        if token.isalpha():
            suggestions = sym_spell.lookup(token.lower(), Verbosity.CLOSEST, max_edit_distance=2)
            corrected_word = suggestions[0].term if suggestions else token
            corrected.append(corrected_word)
        else:
            corrected.append(token)
    return ' '.join(corrected).replace(" .", ".")

# تست
test_sentence = "Th quik braown foxs jmups ovem te lagzy qog."
print(symspell_correct_sentence(test_sentence))


the quirk brown foxes jumps ovum te lazy log.


# OpenAI API (پولی - آنلاین) 80-100%

In [ ]:
import openai

openai.api_key = "My_API_KEY"

def gpt_correct_sentence(sentence):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a grammar correction assistant."},
            {"role": "user", "content": f"Please correct this sentence: {sentence}"}
        ],
        temperature=0
    )
    return response['choices'][0]['message']['content'].strip()

# تست
test_sentence = "Th quik braown foxs jmups ovem te lagzy qog."
print(gpt_correct_sentence(test_sentence))


# BERT + Gramformer (آفلاین)   90%

In [19]:
!pip install gramformer
!pip install torch transformers


ERROR: Could not find a version that satisfies the requirement gramformer (from versions: none)
ERROR: No matching distribution found for gramformer
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 31.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found

In [21]:
!pip install language-tool-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.8 MB/s eta 0:00:00


In [23]:
# from gramformer import Gramformer

# gf = Gramformer(models=1)  # 1 = Corrector

# def bert_correct_sentence(sentence):
#     corrected = list(gf.correct(sentence, max_candidates=1))
#     return corrected[0] if corrected else sentence

# # تست
# print(bert_correct_sentence("Th quik braown foxs jmups ovem te lagzy qog."))

import language_tool_python

tool = language_tool_python.LanguageTool('en-US')

text = "Th quik braown foxs jmups ovem te lagzy qog."
matches = tool.check(text)
corrected_text = language_tool_python.utils.correct(text, matches)

print(corrected_text)


INFO:language_tool_python.download_lt:Unzipping /tmp/tmpbvxv54mh.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-6.5.zip to /root/.cache/language_tool_python.


The quick brown foxes jumps over the lazy dog.


# Edit Distance + Context Score %الگوریتم  60

### ترکیب فاصله ویرایشی و بررسی موقعیت کلمه در جمله

### کلمات درست‌تر با توجه به جایگاهشون در جمله انتخاب

In [27]:
import re
from collections import Counter
from difflib import SequenceMatcher

# Re-load Vocabulary file
vocab_path = "/content/Vocabulary.txt"
with open(vocab_path, "r") as file:
    vocabulary = set(word.strip().lower() for word in file.readlines() if word.strip())

# Corrupted sentence to test
sentence = "Th quik braown foxs jmups ovem te lagzy qog."

# Function to calculate similarity ratio
def similarity_ratio(a, b):
    return SequenceMatcher(None, a, b).ratio()

# Get best match from vocabulary
def get_best_match(word, vocab, threshold=0.7):
    suggestions = [(v, similarity_ratio(word, v)) for v in vocab]
    suggestions = [s for s in suggestions if s[1] >= threshold]
    suggestions.sort(key=lambda x: -x[1])  # Sort by descending similarity
    return suggestions[0][0] if suggestions else word

# Sentence correction using edit distance + context scoring
def context_based_correction(sentence, vocab):
    tokens = re.findall(r"\b\w+\b|[^\w\s]", sentence)
    corrected_tokens = []
    for token in tokens:
        if token.isalpha():
            best = get_best_match(token.lower(), vocab)
            corrected_tokens.append(best)
        else:
            corrected_tokens.append(token)
    return ' '.join(corrected_tokens).replace(" .", ".")

# Apply the algorithm
corrected_sentence = context_based_correction(sentence, vocabulary)
corrected_sentence


'th quirk 1 brown 1 foxes 1 jmups ovem te lazy 1 qog.'

# Custom Weighted Scoring الگوریتم

###ترکیبی از:

###شباهت نگارشی (edit distance) - طول کلمه - شروع و پایان مشابه - تکرار حروف

In [31]:
import re
from difflib import SequenceMatcher
from collections import defaultdict

# Load and clean vocabulary (ASCII only)
vocab_path = "/content/Vocabulary.txt"
with open(vocab_path, "r", encoding="utf-8") as file:
    vocabulary = set(word.strip().lower() for word in file if word.strip() and word.strip().isascii())

# --- COMPONENT 1: Edit Distance similarity ---
def similarity_ratio(a, b):
    return SequenceMatcher(None, a, b).ratio()

def get_best_match_edit(word, vocab, threshold=0.7):
    suggestions = [(v, similarity_ratio(word, v)) for v in vocab]
    suggestions = [s for s in suggestions if s[1] >= threshold]
    suggestions.sort(key=lambda x: -x[1])
    return suggestions[0][0] if suggestions else word

# --- COMPONENT 2: Phonetic Matching (Soundex) ---
def soundex(word):
    word = word.upper()
    if not word or not word.isascii():
        return ""
    replacements = {
        "BFPV": '1', "CGJKQSXZ": '2', "DT": '3',
        "L": '4', "MN": '5', "R": '6'
    }
    result = word[0]
    mapping = {char: code for group, code in replacements.items() for char in group}
    for char in word[1:]:
        code = mapping.get(char, '')
        if code != result[-1]:
            result += code
    result = re.sub(r'[^A-Z0-9]', '', result)
    return (result + '000')[:4]

def get_best_match_phonetic(word, vocab):
    token_sound = soundex(word)
    matches = [v for v in vocab if soundex(v) == token_sound]
    return matches[0] if matches else word

# --- FINAL HYBRID CORRECTION FUNCTION ---
def hybrid_correction(sentence, vocab, edit_threshold=0.7):
    tokens = re.findall(r"\b\w+\b|[^\w\s]", sentence)
    corrected_tokens = []

    for token in tokens:
        if token.isalpha():
            token_lower = token.lower()

            # Try Edit Distance first
            edit_match = get_best_match_edit(token_lower, vocab, threshold=edit_threshold)

            # If edit match is too far, try phonetic
            if edit_match == token_lower:
                phonetic_match = get_best_match_phonetic(token_lower, vocab)
                corrected_tokens.append(phonetic_match)
            else:
                corrected_tokens.append(edit_match)
        else:
            corrected_tokens.append(token)

    return ' '.join(corrected_tokens).replace(" .", ".")

# Test on sample input
test_sentence = "Th quik braown foxs jmups ovem te lagzy qog."
hybrid_result = hybrid_correction(test_sentence, vocabulary)
hybrid_result


'tow 1 quirk 1 brown 1 foxes 1 jumps 1 oven 1 tow 1 lazy 1 quack 1.'

# Phonetic Matching (Soundex یا Metaphone) الگوریتم %10

###تصحیح بر اساس صدا، نه صرفاً نوشتار. مثل وقتی می‌نویسی fone به‌جای phone.



In [25]:
!pip install fuzzy


  Preparing metadata (setup.py) ... done
  Created wheel for fuzzy: filename=Fuzzy-1.2.2-cp311-cp311-linux_x86_64.whl size=220703 sha256=97c598d1997673d7565fa3e8ac29bdba1348b8986a120460ed3373fee53d4850
  Stored in directory: /root/.cache/pip/wheels/c7/1c/77/28af87176ebf6eb6208c17e64a45a8e48eda4194bd8f605096
Successfully built fuzzy


In [29]:
import re
from difflib import SequenceMatcher

# Reload vocabulary file
vocab_path = "/content/Vocabulary.txt"
with open(vocab_path, "r", encoding="utf-8") as file:
    vocabulary = set(word.strip().lower() for word in file.readlines() if word.strip() and word.strip().isascii())

# Improved phonetic correction using Soundex with ASCII-safe fallback
def soundex(word):
    word = word.upper()
    if not word.isascii() or not word:
        return ""
    replacements = {
        "BFPV": '1',
        "CGJKQSXZ": '2',
        "DT": '3',
        "L": '4',
        "MN": '5',
        "R": '6'
    }
    result = word[0]
    mapping = {char: code for group, code in replacements.items() for char in group}

    for char in word[1:]:
        code = mapping.get(char, '')
        if code != result[-1]:
            result += code

    result = re.sub(r'[^A-Z0-9]', '', result)  # Remove non-ASCII
    return (result + '000')[:4]

# Apply phonetic correction using our Soundex
def phonetic_correction(sentence, vocab):
    tokens = re.findall(r"\b\w+\b|[^\w\s]", sentence)
    corrected = []
    for token in tokens:
        if token.isalpha():
            token_sound = soundex(token)
            matches = [v for v in vocab if soundex(v) == token_sound]
            best = matches[0] if matches else token
            corrected.append(best)
        else:
            corrected.append(token)
    return ' '.join(corrected).replace(" .", ".")

# Run the correction on the test sentence
test_sentence = "Th quik braown foxs jmups ovem te lagzy qog."
phonetic_result = phonetic_correction(test_sentence, vocabulary)
phonetic_result


'tow 1 quack 1 bernie 1 fuzzy 1 jumps 1 oven 1 tow 1 lucks 1 quack 1.'

In [35]:
# Fixed vocabulary loading: use only the first word in each line (ignore any frequency or trailing data)
import re
from difflib import SequenceMatcher
from collections import Counter

# Properly load only the first word from each line
vocab_path = "/content/Vocabulary.txt"
with open(vocab_path, "r", encoding="utf-8") as file:
    vocab_words = [line.strip().split()[0].lower() for line in file if line.strip() and line.strip()[0].isascii()]
    vocabulary = set(vocab_words)
    word_freq = Counter(vocab_words)

# Component 1: Edit Distance + Probability
def similarity_ratio(a, b):
    return SequenceMatcher(None, a, b).ratio()

def P(word):
    N = sum(word_freq.values())
    return word_freq[word] / N if word in word_freq else 1 / N

def get_best_probable_match(word, vocab, threshold=0.5):
    suggestions = [(v, similarity_ratio(word, v), P(v)) for v in vocab if similarity_ratio(word, v) >= threshold]
    if not suggestions:
        return word
    suggestions.sort(key=lambda x: (-x[1], -x[2]))
    return suggestions[0][0]

# Component 2: Soundex phonetic match
def soundex(word):
    word = word.upper()
    if not word or not word.isascii():
        return ""
    replacements = {
        "BFPV": '1', "CGJKQSXZ": '2', "DT": '3',
        "L": '4', "MN": '5', "R": '6'
    }
    result = word[0]
    mapping = {char: code for group, code in replacements.items() for char in group}
    for char in word[1:]:
        code = mapping.get(char, '')
        if code != result[-1]:
            result += code
    return (result + '000')[:4]

def get_best_match_phonetic(word, vocab):
    token_sound = soundex(word)
    matches = [v for v in vocab if soundex(v) == token_sound]
    return matches[0] if matches else word

# Fallback: Edit Distance Only
def get_best_match_edit(word, vocab, threshold=0.7):
    suggestions = [(v, similarity_ratio(word, v)) for v in vocab if similarity_ratio(word, v) >= threshold]
    suggestions.sort(key=lambda x: -x[1])
    return suggestions[0][0] if suggestions else word

# Final hybrid correction
def hybrid_correction_clean(sentence, vocab, edit_threshold=0.7):
    tokens = re.findall(r"\b\w+\b|[^\w\s]", sentence)
    corrected_tokens = []

    for token in tokens:
        if token.isalpha():
            token_lower = token.lower()

            best = get_best_probable_match(token_lower, vocab, threshold=edit_threshold)
            if best == token_lower:
                best = get_best_match_phonetic(token_lower, vocab)
            if best == token_lower:
                best = get_best_match_edit(token_lower, vocab, threshold=edit_threshold)

            corrected_tokens.append(best)
        else:
            corrected_tokens.append(token)

    return ' '.join(corrected_tokens).replace(" .", ".")

# Test sentence
test_sentence = "Th quik braown foxs jmups ovem te lagzy qog."
final_clean_result_fixed = hybrid_correction_clean(test_sentence, vocabulary)
final_clean_result_fixed


'the quirk brawn foxes jumps hove tie lazy quashes.'

In [37]:
import re
from collections import defaultdict, Counter
from difflib import get_close_matches

# Load vocabulary (keep as requested)
vocab_path = "/content/Vocabulary.txt"
with open(vocab_path, "r", encoding="utf-8") as f:
    vocab_words = [line.strip().split()[0].lower() for line in f if line.strip() and line[0].isascii()]
    vocabulary = set(vocab_words)

# Build n-gram model (unigram + bigram + trigram)
def build_ngram_model(vocab_list):
    unigram = Counter(vocab_list)
    bigram = defaultdict(lambda: defaultdict(int))
    trigram = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

    for i in range(len(vocab_list) - 2):
        w1, w2, w3 = vocab_list[i], vocab_list[i+1], vocab_list[i+2]
        unigram[w1] += 1
        bigram[w1][w2] += 1
        trigram[w1][w2][w3] += 1

    # Add final two for completeness
    if len(vocab_list) >= 2:
        bigram[vocab_list[-2]][vocab_list[-1]] += 1
        unigram[vocab_list[-1]] += 1

    return unigram, bigram, trigram

unigram_model, bigram_model, trigram_model = build_ngram_model(vocab_words)

# N-gram scoring function
def ngram_score(words):
    score = 0
    for i in range(len(words)):
        if i >= 2:
            w1, w2, w3 = words[i-2], words[i-1], words[i]
            score += trigram_model[w1][w2].get(w3, 0) * 3
        elif i == 1:
            w1, w2 = words[i-1], words[i]
            score += bigram_model[w1].get(w2, 0) * 2
        else:
            w1 = words[i]
            score += unigram_model.get(w1, 0)
    return score

# Generate correction candidates using difflib
def generate_candidates(word, vocab, n=5, cutoff=0.75):
    return get_close_matches(word, vocab, n=n, cutoff=cutoff)

# Final context-aware correction function (with enhanced n-gram model)
def context_aware_ngram_correction(sentence, vocab):
    tokens = re.findall(r"\b\w+\b|[^\w\s]", sentence)
    corrected_tokens = []

    for i, token in enumerate(tokens):
        if token.isalpha():
            token_lower = token.lower()
            candidates = generate_candidates(token_lower, vocab, n=5, cutoff=0.75)
            if not candidates:
                corrected_tokens.append(token_lower)
                continue

            # Try each candidate and pick the best based on context-aware n-gram score
            best_candidate = candidates[0]
            best_score = -1

            for cand in candidates:
                # Build temporary sentence
                trial_tokens = corrected_tokens + [cand] + tokens[i+1:]
                trial_words = [t for t in trial_tokens if t.isalpha()]
                score = ngram_score(trial_words)
                if score > best_score:
                    best_candidate = cand
                    best_score = score

            corrected_tokens.append(best_candidate)
        else:
            corrected_tokens.append(token)

    return ' '.join(corrected_tokens).replace(" .", ".")

# Run the upgraded n-gram correction
test_sentence = "Th quik braown foxs jmups ovem te lagzy qog."
enhanced_ngram_result = context_aware_ngram_correction(test_sentence, vocabulary)
enhanced_ngram_result


'the quirk brown foxes mumps wove te lazy qog.'